# <font color="Blue">Abstract</font>

In this notebook, we delve into the utilization of LangChain and its 'prompt template' function to create prompt for generating cover letters.

**<font color="red">Our goal</font>** is to craft an effective prompt tailored to a cover letter in the context of specific jobs and companies.  

We used following knowledge:  
Prompt Engineering Skills:  
1. Placeholder;
2. Persona.


<font color="green"> Conclusion: </font>  
In this notebook, we explored two prompt: prompt 1 and prompt 2 (prompt 1 with persona). In my opinion, both prompt have good answer.


**Next Step**  
- [ ] Ask help from career coach to get professionals' perspective

# Prepare the notebook for use

## Install packages & import libraries

In [ ]:
# Download required packages to Colab
!pip install -q langchain
!pip install -q openai
!pip install -q chromadb
!pip install -q tiktoken
!pip install -q duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.8/496.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 12.7 MB/s e

In [ ]:
# Import required libraries
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain, RetrievalQA
from langchain import ConversationChain
from langchain.agents import load_tools, initialize_agent, AgentType
from langchain.document_loaders import TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.tools import DuckDuckGoSearchRun
from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
import os
from getpass import getpass

from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.schema import HumanMessage, AIMessage, SystemMessage

import pandas as pd
import csv

## OpenAI API key

In [ ]:
openai_api_key = getpass()
os.environ["OPENAI_API_KEY"] = openai_api_key

··········


## Connect Google Drive

## Create .csv files to collect info from this notebook

Since the results we got from LLMs will be different everytime, we want to use .csv files to track those results which could help use go back to see what we can do or what we did to improve the outputs.

In [ ]:
# with open('/content/drive/MyDrive/Capstone Project - Mock Interview App/variable_combination.csv', 'w', newline='') as csvfile:
#     fieldnames = ['comb_num','interviewer_role','company_name','resume','role_title','role_description','company_description']
#     writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

#     writer.writeheader()
# var_comb = pd.read_csv("/content/drive/MyDrive/Capstone Project - Mock Interview App/variable_combination.csv")

In [ ]:
with open('/content/drive/MyDrive/Capstone Project - Mock Interview App/cover_letter_prompt_result.csv', 'w', newline='') as csvfile:
    fieldnames = ['model', 'variable_combination', 'template', 'output']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    #writer.writerow({'input': user_input, 'output': result})

In [ ]:
var_comb = pd.read_csv("/content/drive/MyDrive/Capstone Project - Mock Interview App/variable_combination.csv")
prompt_result = pd.read_csv("/content/drive/MyDrive/Capstone Project - Mock Interview App/cover_letter_prompt_result.csv")

In [ ]:
var_comb

,comb_num,interviewer_role,company_name,resume,role_title,role_description,company_description
0,1,recruiter,S&P Global,\nEDUCATION\nVanderbilt University – Data Scie...,Data Engineer - AI/ML (Virtual),\nJob Description\n\nThis is a software develo...,\nOverview\nS&P Global (NYSE: SPGI) provides e...


In [ ]:
prompt_result

,model,variable_combination,template,output


## Define Parameters

### 3 interview questions

In [ ]:
var_comb_num = 1

In [ ]:
file = open("/content/drive/MyDrive/Capstone Project - Mock Interview App/data/cover_letter_outline.txt", "r")
cover_letter_outline = file.read()

In [ ]:
print(cover_letter_outline)

Develop a Cover Letter Template
1. Create Adaptable Template
a. Basic Structure: Establish a clear and concise structure with an introduction, body, and conclusion.
b. Personalize the Opening: Prepare a flexible opening that can be customized to address the specific hiring manager or team.
c. Leave Space for Role-Specific Details: Design sections that can be easily modified to reflect the specific job and company.
2. Incorporate Key Skills and Experiences
a. Highlight Transferable Skills: Emphasize skills that are valuable across various roles and industries.
b. Include Achievements: Add sections to incorporate specific accomplishments relevant to the job.
c. Make It Result-Oriented: Focus on how your skills and experiences can benefit the potential employer.
3. Ensure Easy Customization for Specific Roles
a. Use Modifiable Sections: Create parts of the letter that can be easily changed for different job applications.
b. Align with Job Description: Set up placeholders to insert keyword

# Model selection

In [ ]:
chat = ChatOpenAI(model_name = "gpt-4")

In [ ]:
# record model as ex_model
ex_model = "gpt_4"

# Cover letter prompts

## Prompt 1

In [ ]:
# set up input variables for prompt
input_var_1 = ["role_title", "company_name", "resume",
               "role_description", "company_description"]

In [ ]:
template_1 = """
Generate a cover letter template for {role_title} at {company_name}.

Here is the student's resume:
{resume}

Here are the job description and company background that you might want to consider:
Job Description: {role_description}
Company Background: {company_description}

Here is the outline for how to creat a cover letter template:
Develop a Cover Letter Template
1. Create Adaptable Template
a. Basic Structure: Establish a clear and concise structure with an introduction, body, and conclusion.
b. Personalize the Opening: Prepare a flexible opening that can be customized to address the specific hiring manager or team.
c. Leave Space for Role-Specific Details: Design sections that can be easily modified to reflect the specific job and company.
2. Incorporate Key Skills and Experiences
a. Highlight Transferable Skills: Emphasize skills that are valuable across various roles and industries.
b. Include Achievements: Add sections to incorporate specific accomplishments relevant to the job.
c. Make It Result-Oriented: Focus on how your skills and experiences can benefit the potential employer.
3. Ensure Easy Customization for Specific Roles
a. Use Modifiable Sections: Create parts of the letter that can be easily changed for different job applications.
b. Align with Job Description: Set up placeholders to insert keywords and requirements from the job description.
c. Tailor to Company Culture: Prepare a segment to adapt based on the company’s culture and values as described on their website or social media.
"""

In [ ]:
prompt_1 = PromptTemplate(
    input_variables = input_var_1,
    template = template_1
)

In [ ]:
job_title = var_comb.role_title[0]
job_des = var_comb.role_description[0]
comp_name = var_comb.company_name[0]
comp_des = var_comb.company_description[0]
student_resume = var_comb.resume[0]

### Code

In [ ]:
new_prompt = prompt_1.format(role_title = job_title,
                             company_name = comp_name,
                             resume = student_resume,
                             role_description = job_des,
                             company_description = comp_des)
result = chat([HumanMessage(content=new_prompt)])

In [ ]:
print(result.content)

Dear [Hiring Manager's Name],

I am writing to express my interest in the Data Engineer - AI/ML (Virtual) position at S&P Global. As a Master's student at Vanderbilt University’s Data Science Institute with a Bachelor's degree in Data Science and Mathematics, I have honed my skills in Python, SQL, and R, which align with the key requirements of this role. I am particularly excited about the opportunity to join a globally recognized firm like S&P Global and contribute to your mission of providing essential intelligence for smarter decision-making.

During my tenure at Washington University School of Medicine – Institute for Informatics as a Data Science Intern, I successfully implemented Python-based automation to improve data quality control, reducing document download time by a significant 99.2%. I also optimized database updates using Python and Rave, achieving a 75% time reduction in updating birth records. These experiences have equipped me with a strong understanding of machine le

##### store output

In [ ]:
# ex_model, qa_comb_num, var_comb_num, template_1, result_11.content

new_prompt_result = pd.DataFrame({
    'model': [ex_model],
    'variable_combination': [var_comb_num],
    'template': [template_1],
    'output': [result.content]
})

prompt_result = pd.concat([prompt_result, new_prompt_result], ignore_index=True)

In [ ]:
prompt_result

,model,variable_combination,template,output
0,gpt_4,1,\nGenerate a cover letter template for {role_t...,"Dear [Hiring Manager's Name],\n\nI am writing ..."


## Prompt 2

In [ ]:
template_2 = """
As an experienced career coach, you help students improve their skills in
writing a cover letter to get job interviews.  Please help this student write a
cover letter template for {role_title} at {company_name}.

Here is the student's resume:
{resume}

Here are the job description and company background that you might want to consider:
Job Description: {role_description}
Company Background: {company_description}

Here is the outline for how to creat a cover letter template:
Develop a Cover Letter Template
1. Create Adaptable Template
a. Basic Structure: Establish a clear and concise structure with an introduction, body, and conclusion.
b. Personalize the Opening: Prepare a flexible opening that can be customized to address the specific hiring manager or team.
c. Leave Space for Role-Specific Details: Design sections that can be easily modified to reflect the specific job and company.
2. Incorporate Key Skills and Experiences
a. Highlight Transferable Skills: Emphasize skills that are valuable across various roles and industries.
b. Include Achievements: Add sections to incorporate specific accomplishments relevant to the job.
c. Make It Result-Oriented: Focus on how your skills and experiences can benefit the potential employer.
3. Ensure Easy Customization for Specific Roles
a. Use Modifiable Sections: Create parts of the letter that can be easily changed for different job applications.
b. Align with Job Description: Set up placeholders to insert keywords and requirements from the job description.
c. Tailor to Company Culture: Prepare a segment to adapt based on the company’s culture and values as described on their website or social media.
"""

In [ ]:
prompt_2 = PromptTemplate(
    input_variables = input_var_1,
    template = template_2
)

### Code

In [ ]:
new_prompt = prompt_2.format(role_title = job_title,
                             company_name = comp_name,
                             resume = student_resume,
                             role_description = job_des,
                             company_description = comp_des)
result = chat([HumanMessage(content=new_prompt)])

In [ ]:
print(result.content)

Dear [Hiring Manager's Name],

I am writing to express my interest in the Data Engineer - AI/ML (Virtual) position at S&P Global. As a Masters student in Data Science from Vanderbilt University with a strong interest in AI and machine learning, I believe I am well-suited to contribute to your team. I am excited about the prospect of working with S&P Global, a company that is committed to providing essential intelligence to power decision-making.

My academic and professional experiences have equipped me with a strong foundation in data science, including coursework in Data Science Algorithms, Machine Learning, and practical applications in industry. During my internship at the Washington University School of Medicine, I successfully implemented Python-based automation for data quality control, reducing document download time by 99.2%. I also leveraged Python and Rave to optimize database updates, achieving a 75% time reduction. These experiences have honed my skills in software develop

##### store output

In [ ]:
# ex_model, qa_comb_num, var_comb_num, template_1, result_11.content

new_prompt_result = pd.DataFrame({
    'model': [ex_model],
    'variable_combination': [var_comb_num],
    'template': [template_2],
    'output': [result.content]
})

prompt_result = pd.concat([prompt_result, new_prompt_result], ignore_index=True)

In [ ]:
prompt_result

,model,variable_combination,template,output
0,gpt_4,1,\nGenerate a cover letter template for {role_t...,"Dear [Hiring Manager's Name],\n\nI am writing ..."
1,gpt_4,1,"\nAs an experienced career coach, you help stu...","Dear [Hiring Manager's Name],\n\nI am writing ..."


# Save data

In [ ]:
#prompt_result
prompt_result.to_csv("/content/drive/MyDrive/Capstone Project - Mock Interview App/cover_letter_prompt_result.csv", index=False)

In [ ]:
prompt_result

,model,variable_combination,template,output
0,gpt_4,1,\nGenerate a cover letter template for {role_t...,"Dear [Hiring Manager's Name],\n\nI am writing ..."
1,gpt_4,1,"\nAs an experienced career coach, you help stu...","Dear [Hiring Manager's Name],\n\nI am writing ..."


# **Continue at 1_.ipynb**